In [1]:
!pip install mlflow boto3 awscli optuna xgboost imbalanced-learn

In [2]:
import mlflow
import os
# Step 2: Set up the MLflow tracking server
mlflow.set_tracking_uri(os.getenv("MLFLOW_SERVER_TRACKING_URI_EC2"))

In [3]:
# Set or create an experiment
mlflow.set_experiment("Exp 5 - ML Algos with HP Tuning")

<Experiment: artifact_location='s3://youtube-sentimental-bucket/649374757870029171', creation_time=1752677737263, experiment_id='649374757870029171', last_update_time=1752677737263, lifecycle_stage='active', name='Exp 5 - ML Algos with HP Tuning', tags={}>

In [4]:
import pandas as pd
import os
import mlflow
import mlflow.sklearn
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.feature_extraction.text import TfidfVectorizer
from imblearn.over_sampling import SMOTE
from xgboost import XGBClassifier
import optuna


In [5]:
df = pd.read_csv(r'D:\bybappy\youtube_sentimental_analyzer\notebooks\artifacts\dataset.csv').dropna()
df.shape

(36662, 2)

In [6]:


# Create artifacts directory if not present
os.makedirs("artifacts", exist_ok=True)

# Step 1: Remap class labels
df['category'] = df['category'].map({-1: 2, 0: 0, 1: 1})
df = df.dropna(subset=['category'])  # Remove rows with missing target

# Step 2: TF-IDF Vectorization
ngram_range = (1, 3)
max_features = 10000

X_train, X_test, y_train, y_test = train_test_split(
    df['clean_comment'], df['category'], test_size=0.2, random_state=42, stratify=df['category'])

vectorizer = TfidfVectorizer(ngram_range=ngram_range, max_features=max_features)
X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)

# Step 3: Apply SMOTE only on training data
smote = SMOTE(random_state=42)
X_train_vec, y_train = smote.fit_resample(X_train_vec, y_train)

# Step 4: MLflow logging function
def log_mlflow(model_name, model, X_train, X_test, y_train, y_test):
    with mlflow.start_run():
        mlflow.set_tag("mlflow.runName", f"{model_name}_SMOTE_TFIDF_Trigrams")
        mlflow.set_tag("experiment_type", "algorithm_comparison")
        mlflow.log_param("algo_name", model_name)

        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)

        accuracy = accuracy_score(y_test, y_pred)
        mlflow.log_metric("accuracy", accuracy)

        report = classification_report(y_test, y_pred, output_dict=True)
        for label, metrics in report.items():
            if isinstance(metrics, dict):
                for metric_name, value in metrics.items():
                    mlflow.log_metric(f"{label}_{metric_name}", value)

        # Confusion matrix plot
        cm = confusion_matrix(y_test, y_pred)
        plt.figure(figsize=(8, 6))
        sns.heatmap(cm, annot=True, fmt="d", cmap="Blues")
        plt.xlabel("Predicted")
        plt.ylabel("Actual")
        plt.title(f"Confusion Matrix: {model_name}")
        cm_path = os.path.join("artifacts", f"conf_matrix_{model_name}.png")
        plt.savefig(cm_path)
        mlflow.log_artifact(cm_path)
        plt.close()

        mlflow.sklearn.log_model(model, f"{model_name}_model")

# Step 5: Optuna objective function for XGBoost
def objective_xgboost(trial):
    n_estimators = trial.suggest_int('n_estimators', 50, 300)
    learning_rate = trial.suggest_float('learning_rate', 1e-4, 1e-1, log=True)
    max_depth = trial.suggest_int('max_depth', 3, 10)

    model = XGBClassifier(n_estimators=n_estimators,
                          learning_rate=learning_rate,
                          max_depth=max_depth,
                          random_state=42,
                          use_label_encoder=False,
                          eval_metric='mlogloss')  # For multi-class

    model.fit(X_train_vec, y_train)
    y_pred = model.predict(X_test_vec)
    return accuracy_score(y_test, y_pred)

# Step 6: Run Optuna optimization and log best model
def run_optuna_experiment():
    study = optuna.create_study(direction="maximize")
    study.optimize(objective_xgboost, n_trials=30)

    best_params = study.best_params
    best_model = XGBClassifier(n_estimators=best_params['n_estimators'],
                               learning_rate=best_params['learning_rate'],
                               max_depth=best_params['max_depth'],
                               random_state=42,
                               use_label_encoder=False,
                               eval_metric='mlogloss')

    log_mlflow("XGBoost", best_model, X_train_vec, X_test_vec, y_train, y_test)

# Run the experiment
run_optuna_experiment()


[I 2025-07-16 21:43:27,216] A new study created in memory with name: no-name-f003e48a-b273-4f88-86d3-9ed2978e3482
C:\Users\btirk\AppData\Roaming\Python\Python311\site-packages\xgboost\training.py:183: UserWarning: [21:43:27] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
[I 2025-07-16 21:51:49,071] Trial 0 finished with value: 0.6395745261148233 and parameters: {'n_estimators': 282, 'learning_rate': 0.0023814704322778985, 'max_depth': 10}. Best is trial 0 with value: 0.6395745261148233.
C:\Users\btirk\AppData\Roaming\Python\Python311\site-packages\xgboost\training.py:183: UserWarning: [21:51:49] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
[I 2025-07-16 21:54:32,540] Trial 1 finished with value: 0.7419882721941906 and parameters: {'n_estimators': 182, 